In [15]:
# !pip install monai
# !pip install nibabel

     |████████████████████████████████| 650 kB 12.1 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [1]:
import os
import SimpleITK as sitk
import numpy as np
import glob
import matplotlib.pyplot as plt
import pandas as pd

import nibabel
from monai.transforms import Spacing
from monai.transforms import SpatialPad
from monai.transforms import Resize
from monai.transforms import CropForeground

import time
import torch

In [2]:
!pwd

/workspace/JO_nb/Preprocessing


In [3]:
import sys
sys.path.insert(1, '/workspace')
from utils.utils import *

In [4]:
os.cpu_count()

48

# Load Image and Label

In [7]:
raw_data_dir = '/workspace/BoneMeta_raw'

image_data_dir = '/workspace/BoneMeta_new/images'
label_data_dir = '/workspace/BoneMeta_new/labels'

all_1mm_image_dir = '/workspace/BoneMeta_all_1mm/images'
all_1mm_label_dir = '/workspace/BoneMeta_all_1mm/labels'

z_1mm_image_dir = '/workspace/BoneMeta_z_1mm/images'
z_1mm_label_dir = '/workspace/BoneMeta_z_1mm/labels'

data_info_path = '/workspace/DataInfo/data_info_V_2022_04_27.csv'


def get_label_path(file):
    return os.path.join(label_data_dir, file)

def get_image_path(file):
    return os.path.join(image_data_dir, file)

In [8]:
data_info = pd.read_csv(data_info_path, index_col=0)

In [10]:
pd.read_csv(data_info_path,index_col=0).tail()

,Case,Lesion Type,Lesion Location,Probability,I/V,File Name,Lesion Size,Background Size,Original Shape,1mm Resized Shape,...,1mm Resized Center,Pixel Size,Spine Shape,Spine Resized Shape,Spine Center,Spine Resized Shape.1,X Spacing,Y Spacing,Slice Interval,Volume
2093,KH039_20210301_Chest_(-)_Routine,L,T8,100,V,lesionAnnot3D-011.nii.gz,11.2,48036.8,"[512, 512, 327]","[372, 372, 409]",...,"[191, 227, 214]",16965,"[256, 256, 327]","[45, 134, 214]","[63, 185, 171]","[45, 134, 214]",0.7266,0.7266,-1.25,11195.787944
2094,KH040_20210301_Chest(+),L,C7,100,V,lesionAnnot3D-000.nii.gz,2602.3,31483.7,"[512, 512, 349]","[320, 320, 349]",...,"[172, 163, 11]",6662,"[256, 256, 349]","[47, 83, 11]","[75, 133, 11]","[47, 83, 11]",0.6250,0.6250,-1.00,2602.343750
2095,KH042_20210501_Chest(+),L,"T4,5",100,V,lesionAnnot3D-001.nii.gz,35.4,448412.6,"[512, 512, 343]","[308, 308, 343]",...,"[171, 219, 94]",97946,"[256, 256, 343]","[51, 142, 94]","[85, 237, 94]","[51, 142, 94]",0.6016,0.6016,-1.00,35448.867062
2096,KH043_20210601_Thorax^00_Fl_Chest_Routine_(Adult),L,T3,100,V,lesionAnnot3D-001.nii.gz,13.1,98644.9,"[512, 512, 331]","[367, 367, 331]",...,"[185, 203, 56]",25516,"[256, 256, 331]","[42, 112, 56]","[59, 156, 56]","[42, 112, 56]",0.7168,0.7168,-1.00,13110.177956
2097,KH043_20210601_Thorax^00_Fl_Chest_Routine_(Adult),L,Rt 6th rib,100,V,lesionAnnot3D-002.nii.gz,99.7,1186700.3,"[512, 512, 331]","[367, 367, 331]",...,"[53, 178, 183]",194109,"[256, 256, 331]","[-90, 86, 183]","[-126, 121, 183]","[-90, 86, 183]",0.7168,0.7168,-1.00,99733.639004


In [11]:
data_info.head()

,Case,Lesion Type,Lesion Location,Probability,I/V,File Name,Lesion Size,Background Size,Original Shape,1mm Resized Shape,...,1mm Resized Center,Pixel Size,Spine Shape,Spine Resized Shape,Spine Center,Spine Resized Shape.1,X Spacing,Y Spacing,Slice Interval,Volume
0,000232_20190401_CT_Abdomen+Pelvis_Dynamic_(con...,S,T10-1,metastasis,V,lesionAnnot3D-000.nii.gz,731.0,7589.0,"[512, 512, 435]","[271, 271, 435]",...,"[118, 158, 42]",2590,"[256, 256, 435]","[12, 90, 42]","[24, 170, 42]","[12, 90, 42]",0.5312,0.5312,-1.0,730.829210
1,000232_20190401_CT_Abdomen+Pelvis_Dynamic_(con...,S,T11,metastasis,V,lesionAnnot3D-002.nii.gz,1635.2,12690.8,"[512, 512, 435]","[271, 271, 435]",...,"[129, 181, 65]",5794,"[256, 256, 435]","[23, 113, 65]","[44, 213, 65]","[23, 113, 65]",0.5312,0.5312,-1.0,1634.912911
2,000232_20190401_CT_Abdomen+Pelvis_Dynamic_(con...,S,T12,metastasis,V,lesionAnnot3D-003.nii.gz,570.1,8361.9,"[512, 512, 435]","[271, 271, 435]",...,"[122, 155, 83]",2020,"[256, 256, 435]","[16, 87, 83]","[30, 165, 83]","[16, 87, 83]",0.5312,0.5312,-1.0,569.990349
3,000232_20190401_CT_Abdomen+Pelvis_Dynamic_(con...,S,L1-1,metastasis,V,lesionAnnot3D-004.nii.gz,3222.2,54917.8,"[512, 512, 435]","[271, 271, 435]",...,"[139, 149, 117]",11417,"[256, 256, 435]","[33, 81, 117]","[62, 154, 117]","[33, 81, 117]",0.5312,0.5312,-1.0,3221.574164
4,000232_20190401_CT_Abdomen+Pelvis_Dynamic_(con...,S,L1-2,metastasis,V,lesionAnnot3D-005.nii.gz,921.8,10446.2,"[512, 512, 435]","[271, 271, 435]",...,"[145, 145, 128]",3266,"[256, 256, 435]","[39, 77, 128]","[73, 145, 128]","[39, 77, 128]",0.5312,0.5312,-1.0,921.578455


In [14]:
ct_info = data_info.loc[:,['Case', '1mm Resized Shape', 'Original Shape', 'X Spacing', 'Y Spacing', 'Slice Interval']]
ct_info.drop_duplicates(inplace=True)
ct_info.set_index('Case',inplace=True)
print(ct_info.shape)
ct_info.head()

(318, 5)


,1mm Resized Shape,Original Shape,X Spacing,Y Spacing,Slice Interval
Case,,,,,
000232_20190401_CT_Abdomen+Pelvis_Dynamic_(contrast),"[271, 271, 435]","[512, 512, 435]",0.5312,0.5312,-1.0
000232_20190423_Chest_CT_(contrast),"[348, 348, 315]","[512, 512, 315]",0.6816,0.6816,-1.0
000285_20190801_CT_Abdomen+Pelvis_Pre-Post_(contrast),"[350, 350, 528]","[512, 512, 528]",0.6836,0.6836,-0.8
000285_20190807_Chest_CT_(contrast),"[306, 306, 238]","[512, 512, 238]",0.5977,0.5977,-1.0
000316_20190627_Chest_CT_(contrast),"[293, 293, 283]","[512, 512, 283]",0.5742,0.5742,-1.0


In [35]:
# 50% info 
# X Spacing: 0.63
# 1mm size: 323, 323, 402 

count    318.000000
mean       0.643018
std        0.072318
min        0.365200
25%        0.603500
50%        0.630900
75%        0.677700
max        1.269500
Name: X Spacing, dtype: float64

In [10]:
ct_info.loc['000350_20190927_chest'].resolution

'[0.589844, 0.589844, 1.0]'

In [30]:
ct_info.loc['000350_20190927_chest'].dimension

'[512, 512, 327]'

In [21]:
def resolution_to_crz_float(resolution):
    crz = resolution.strip('[]').split(', ')
    return [float(value) for value in crz]

def resample_all_1mm(np_image, resolution):
    crz = resolution_to_crz_float(resolution)
    pixdim = [1/crz[2], 1/crz[1], 1/crz[0]] # crz to zrc
    spacer = Spacing(pixdim, dtype=np.float64)
    spaced_image = spacer(np_image[np.newaxis,:]) # with channel
    return spaced_image[0][0] #in tuple, without channel


def resample_z_1mm(np_image, resolution):
    crz = resolution_to_crz_float(resolution)
    pixdim = [1/crz[2], 1, 1] # crz to zrc
    spacer = Spacing(pixdim, dtype=np.float64)
    spaced_image = spacer(np_image[np.newaxis,:]) # with channel
    return spaced_image[0][0] #in tuple, without channel

In [21]:
def save_image(np_image, target_dir, file_name):
    np_image = np_image.astype(np.int16)
    np.save(os.path.join(target_dir, file_name), np_image)
    
def save_label(np_image, target_dir, file_name):
    np_image = (np_image>0.5).astype(np.uint8)
    np.save(os.path.join(target_dir, file_name), np_image)

In [9]:
def resample_all_files(file_dir, target_dir, resample_func, save_func):
    file_list = os.listdir(file_dir)
    for file in file_list:
        np_file = np.load(os.path.join(file_dir, file))
        resolution = ct_info.loc[file_to_case(file)].resolution        
        resampled_image = resample_func(np_file, resolution)
        save_func(resampled_image, target_dir, file)

In [99]:
os.makedirs(all_1mm_image_dir, exist_ok=True)
os.makedirs(all_1mm_label_dir, exist_ok=True)
os.makedirs(z_1mm_image_dir, exist_ok=True)
os.makedirs(z_1mm_label_dir, exist_ok=True)

In [100]:
t = time.perf_counter()

resample_all_files(image_data_dir, all_1mm_image_dir, resample_all_1mm, save_image)

elapsed_time = time.perf_counter() - t

print(f'''
consumed_time: {elapsed_time}s
''')


consumed_time: 939.9781629740028s



In [102]:
np.load(os.path.join(all_1mm_image_dir,'SN059_20170501_chest.npy')).shape

(328, 424, 424)

In [112]:
resample_all_files(label_data_dir, all_1mm_label_dir, resample_all_1mm, save_label)

In [104]:
np.load(os.path.join(all_1mm_label_dir,'SN059_20170501_chest.npy')).shape

(328, 424, 424)

In [105]:
resample_all_files(image_data_dir, z_1mm_image_dir, resample_z_1mm, save_image)

In [113]:
resample_all_files(label_data_dir, z_1mm_label_dir, resample_z_1mm, save_label)

## Resample all same size

In [36]:
!ls /workspace

3dunet		     BoneMeta_all_1mm  BoneSeg_CropSpine  JS_nb   utils
BoneMeta_all_128     BoneMeta_new      BoneSeg_all_128	  MONAI
BoneMeta_all_192     BoneMeta_raw      DataInfo		  jeune
BoneMeta_all_192_VI  BoneSeg	       JO_nb		  jungoh


In [37]:
from multiprocessing import Pool

class Resampler:
    def __init__(self, origin_dir, target_dir, resample_size, is_label=False):
        self.origin_dir, self.target_dir, self.resample_size, self.is_label = origin_dir, target_dir, resample_size, is_label

    def save_np(self, np_image, target_dir, file_name):
        if self.is_label:
            np_image = (np_image>0.5).astype(np.uint8)            
        else:
            np_image = np_image.astype(np.int16)
        np.save(os.path.join(target_dir, file_name), np_image)

    def resize(self, np_image): 
        if self.is_label:
            np_image = np_image > 0.5
        resizer = Resize(self.resample_size)
        resized_image = resizer(np_image[np.newaxis,:]) # with channel
        return resized_image[0] # without channel

    def resample(self, file, origin_dir, target_dir, resample_func, save_func):
        np_file = np.load(os.path.join(origin_dir, file))
        resampled_image = resample_func(np_file)
        save_func(resampled_image, target_dir, file)

    def wrapper(self, args):
        return self.resample(*args)

    def resample_all(self):
        os.makedirs(self.target_dir, exist_ok=True)

        t = time.perf_counter()

        file_list = os.listdir(self.origin_dir)
        arg_list = [(file, self.origin_dir, self.target_dir, self.resize, self.save_np) for file in file_list]
        pool = Pool(processes=32)
        pool.map(self.wrapper, arg_list)
        pool.close()
        pool.join()

        elapsed_time = time.perf_counter() - t

        print(f'''
        consumed_time: {elapsed_time}s
        ''')

class MultiChResampler:
    def __init__(self, origin_dir, target_dir, resample_size, is_label=False):
        self.origin_dir, self.target_dir, self.resample_size, self.is_label = origin_dir, target_dir, resample_size, is_label

    def save_np(self, np_image, target_dir, file_name):
        if self.is_label:
            np_image = np_image.astype(np.uint8)            
        else:
            np_image = np_image.astype(np.int16)
        np.save(os.path.join(target_dir, file_name), np_image)

    def resize(self, np_image): 
        if self.is_label:
            resizer = Resize(self.resample_size, mode='nearest')
        else: 
            resizer = Resize(self.resample_size)
        resized_image = resizer(np_image[np.newaxis,:]) # with channel
        return resized_image[0] # without channel

    def resample(self, file, origin_dir, target_dir, resample_func, save_func):
        np_file = np.load(os.path.join(origin_dir, file))
        resampled_image = resample_func(np_file)
        save_func(resampled_image, target_dir, file)

    def wrapper(self, args):
        return self.resample(*args)

    def resample_all(self):
        os.makedirs(self.target_dir, exist_ok=True)

        t = time.perf_counter()

        file_list = os.listdir(self.origin_dir)
        arg_list = [(file, self.origin_dir, self.target_dir, self.resize, self.save_np) for file in file_list]
        pool = Pool(processes=32)
        pool.map(self.wrapper, arg_list)
        pool.close()
        pool.join()

        elapsed_time = time.perf_counter() - t

        print(f'''
        consumed_time: {elapsed_time}s
        ''')

In [39]:
!ls /workspace/BoneMeta_new/images

'000010_20181214_CT_Abdomen+Pelvis_Dynamic_(contrast).npy'
'000011_20181207_CT_Abdomen+Pelvis_Post_(contrast).npy'
'000011_20181207_Chest_CT_(contrast).npy'
'000019_20181018_Chest_CT_(contrast).npy'
'000019_20190601_CT_Abdomen+Pelvis_Dynamic_(contrast).npy'
'000019_20190613_Chest_CT_(contrast).npy'
'000021_20181227_Chest_CT_(contrast).npy'
'000022_20180830_Chest_CT_(contrast).npy'
'000025_20180808_CT_Abdomen+Pelvis_Dynamic_(contrast).npy'
'000025_20180808_Chest_CT_(contrast).npy'
'000048_20190501_CT_Abdomen+Pelvis_Uro_(contrast).npy'
'000069_20180319_Chest_CT_(contrast).npy'
'000079_20180911_Pulmonary_artery_CT_Angio+3D_(contrast).npy'
'000080_20180911_Chest_CT_(contrast).npy'
'000085_20180829_Chest_CT_(contrast).npy'
'000162_20180131_Chest_CT_(contrast).npy'
'000193_20190114_Chest_CT_(contrast).npy'
'000212_20190324_Chest_CT_(noncontrast).npy'
'000224_20190228_Chest_CT_(contrast).npy'
'000232_20190401_CT_Abdomen+Pelvis_Dynamic_(contrast).npy'
'000232_20190423_Chest_CT_(contrast).npy'


In [41]:
source_dir = '/workspace/BoneMeta_new/images'
target_dir = '/workspace/BoneMeta_all_256/images'

source_dir_lb = '/workspace/BoneMeta_new/labels'
target_dir_lb = '/workspace/BoneMeta_all_256/labels'

image_resampler = Resampler(source_dir, target_dir, [256,256,256])
label_resampler_1 = Resampler(source_dir_lb, target_dir_lb, [256,256,256], True)
# label_resampler_2 = Resampler('/workspace/BoneMeta_new/labels_mixed', '/workspace/BoneMeta_all_192/labels_mixed', [192,192,192], True)
# label_resampler_3 = Resampler('/workspace/BoneMeta_new/labels_sclerotic', '/workspace/BoneMeta_all_192/labels_sclerotic', [192,192,192], True)

image_resampler.resample_all()
label_resampler_1.resample_all()
# label_resampler_2.resample_all()
# label_resampler_3.resample_all()



        consumed_time: 96.9624547273852s
        

        consumed_time: 22.2795384619385s
        


In [95]:
np.load('/workspace/BoneMeta_all_128/images/BH008_20190701_abdomen.npy').shape

(128, 128, 128)

In [10]:
file_names = os.listdir(all_128_image_dir)
image_sizes = [np.load(os.path.join(all_128_image_dir, file)).shape for file in file_names]
label_sizes = [np.load(os.path.join(all_128_label_dir, file)).shape for file in file_names]

In [11]:
label_sizes

[(128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128

# Test Codes

## Testing One Case

In [64]:
img = np.load(get_image_path('000251_20190703_chest.npy'))

In [73]:
cropper = CropForeground()

In [78]:
cropper.compute_bounding_box(img[np.newaxis,:])

(array([ 0, 58,  0]), array([297, 512, 512]))

In [79]:
cropper(img).shape

(297, 454, 512)

In [89]:
import time

In [90]:
t = time.perf_counter()

resampled_image = resample_all_1mm(img, ct_info.resolution[0])

elapsed_time = time.perf_counter() - t

print(f'''
consumed_time: {elapsed_time}s
''')


consumed_time: 1.6059333749872167s



In [53]:
img.shape

(297, 512, 512)

In [54]:
resampled_image.shape

(297, 287, 287)

In [56]:
label = np.load(get_label_path('000251_20190703_chest.npy'))

In [57]:
resampled_label = resample_all_1mm(label, ct_info.resolution[0])

In [77]:
adjust_window(resampled_image, [700,100]).astype(np.int32).min()

0

In [ ]:
plot_image_and_label(adjust_window(resampled_image, [700,100]).astype(np.int32), (resampled_label>0).astype(np.uint8))

In [29]:
from monai.transforms import Rand3DElastic
sample_np_image = dcm_to_numpy(cases[0], dcm_base_dir) # npy 로 변환한 3D 이미지 하나 입니다
sample_label = np.load('/workspace/BoneMeta_new/labels/{}.npy'.format(cases[0]))
elastic_deform = Rand3DElastic(sigma_range=(35,35), magnitude_range=(10,10), prob=1.0, rotate_range=0, shear_range=0)
deformed_image = elastic_deform(sample_np_image)


## Testing All Sizes

In [114]:
file_names = os.listdir(all_1mm_image_dir)

In [115]:
image_sizes = [np.load(os.path.join(all_1mm_image_dir, file)).shape for file in file_names]
label_sizes = [np.load(os.path.join(all_1mm_label_dir, file)).shape for file in file_names]

In [116]:
image_sizes == label_sizes

True

In [ ]:
plot_image_and_label(np.load(os.path.join(all_1mm_image_dir, file_names[5])), np.load(os.path.join(all_1mm_label_dir, file_names[5])))